In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import roc_auc_score
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

import gc
import os
print(os.listdir("input"))


ModuleNotFoundError: No module named 'keras'

In [ ]:
df = pd.read_csv('input/Lender_Raw_Data.csv')
print("Raw shape: ", df.shape)

df.set_index('SK_ID_CURR', inplace=True)

y = df['TARGET'].copy()
feats = [f for f in df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

df.drop(['index', 'TARGET'], axis=1, inplace=True)

print("X shape: ", df.shape, "    y shape:", y.shape)

print("\nPreparing data...")
for feat in feats:
    df[feat] = df[feat].fillna(df[feat].mean())

In [ ]:
# i must congrats someone that did this, but i read it on internet, please if it's you, congrats, and explain your code :)
def rank_gauss(x):
    from scipy.special import erfinv
    N = x.shape[0]
    temp = x.argsort()
    rank_x = temp.argsort() / N
    rank_x -= rank_x.mean()
    rank_x *= 2
    efi_x = erfinv(rank_x)
    efi_x -= efi_x.mean()
    return efi_x

In [ ]:
for i in df.columns:
    #print('Categorical: ',i)
    df[i] = rank_gauss(df[i].values)

In [ ]:
training = y.notnull()
testing = y.isnull()
X_train = df[training].values
X_test = df[testing].values
y_train = np.array(y[training])
print( X_train.shape, X_test.shape, y_train.shape )
gc.collect()

In [1]:
import tensorflow as tf
from keras.callbacks import Callback
import logging
class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=10):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.X_val, verbose=0)
        score = roc_auc_score(self.y_val, y_pred)
        
        logging.info("interval evaluation - epoch: {:d} - score: {:.6f}".format(epoch, score))
        print("interval evaluation - epoch: {:d} - score: {:.6f}".format(epoch, score))
        logs['val_auc'] = score

SyntaxError: invalid syntax (<ipython-input-1-b371b6c6f8ee>, line 17)

In [ ]:
print( 'Saving results...' )
sub = pd.DataFrame()
sub_train = pd.DataFrame()
sub['SK_ID_CURR'] = df[testing].index
sub['TARGET'] = sub_preds
sub[['SK_ID_CURR', 'TARGET']].to_csv('sub_nn.csv', index= False)

print( sub.head() )